<a href="https://colab.research.google.com/github/KalkulatorHere/NN_LLM/blob/main/NN_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df=messages = pd.read_csv('/content/SMSSpamCollection.txt', sep='\t',
                           names=["label", "message"])
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [2]:

df.shape

(5572, 2)

In [3]:

X=list(df['message'])

In [4]:

y=list(df['label'])

In [5]:

y=list(pd.get_dummies(y,drop_first=True)['spam'])

In [6]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [7]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.4 MB/s eta 0:00:00


In [8]:

from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [9]:

train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)


In [10]:

import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))


In [11]:
from transformers import TFTrainingArguments
training_args = TFTrainingArguments(
    output_dir='./results',
    evaluation_strategy="steps",
    eval_steps=100,
    num_train_epochs=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)


In [12]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = TFTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [13]:

trainer.evaluate(test_dataset)

{'eval_loss': 0.020890959671565464}

In [14]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[ 3.383186 , -3.5276804],
       [-2.6005578,  3.1683407],
       [ 3.045614 , -3.1959352],
       ...,
       [ 2.9635873, -3.1317692],
       [-2.6156573,  3.1971674],
       [ 2.717972 , -2.8406467]], dtype=float32), label_ids=array([0, 1, 0, ..., 0, 1, 0], dtype=int32), metrics={'eval_loss': 0.020896947383880614})

In [32]:
trainer.predict(test_dataset)[1].shape

(1115, 2)

In [53]:
output=trainer.predict(test_dataset)[1]

In [54]:

from sklearn.metrics import confusion_matrix

cm=confusion_matrix(y_test,output)
cm

array([[955,   0],
       [  0, 160]])

In [55]:

trainer.save_model('senti_model')

In [56]:
!pip install gradio
import gradio as gr
import tensorflow as tf
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification


In [57]:

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

model = TFDistilBertForSequenceClassification.from_pretrained('/content/senti_model')


Some layers from the model checkpoint at /content/senti_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/senti_model and are newly initialized: ['dropout_139']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [58]:

def predict_sentiment(text_list):
    try:
        your_max_length=10
        inputs = tokenizer(text_list, truncation=True, padding=True, return_tensors="tf", max_length=your_max_length)
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']

        # Predict using the model
        prediction = model.predict([input_ids, attention_mask])[0]
        sentiment_label = "Spam" if prediction[1] > prediction[0] else "Ham"
        confidence = max(prediction)

        # Format the output string
        result = f"Predicted Label: {sentiment_label} (Confidence: {confidence:.2f})"

        return result
    except Exception as e:
        return f"Error: {str(e)}"



iface = gr.Interface(
    fn=predict_sentiment,
    inputs="text",
    outputs="text",
    title="Spam Detection App",
    description="Enter a message to predict if it's spam or not."
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://b3ff9561dcab168a35.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
